# Prompts no LangChain

Quando trata-se de IA generativa de texto, todas as LLMs necessitam de um input. 

O input é o que chamamos de "prompt", que é a frase ou texto que a LLM irá completar. A qualidade do input é crucial para a qualidade da resposta da LLM.

O Langchain possui diversas interfaces para a criação desses templates, que é o que veremos neste notebook.

### LLM utilizada = Gemini-pro

In [3]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())

gemini_api_key = os.getenv("GEMINI_API_KEY")

llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=gemini_api_key, temperature=0)

## Prompt Template

Com esta classe, você consegue criar templates bastante complexos para a geração de prompts de uma maneira simples.

O que necessita é de um template (Uma instrução ou qualquer texto que você deseja que seja preenchido) e de um conjunto de palavras-chave que irão preencher o template.

In [5]:
from langchain.prompts import PromptTemplate

template = """
Você é o melhor programador da linguagem {linguagem}. Você irá auxiliar o usuário à realizar tarefas para esta linguagem 
em específico. Abaixo entre aspas está o comando:


'''{comando}'''
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["linguagem", "comando"]
)

print(prompt.format(
    linguagem="Python",
    comando="Crie uma função onde eu faço uma requisição numa URL com o requests"
))


Você é o melhor programador da linguagem Python. Você irá auxiliar o usuário à realizar tarefas para esta linguagem 
em específico. Abaixo entre aspas está o comando:


'''Crie uma função onde eu faço uma requisição numa URL com o requests'''



In [12]:
resp_llm = llm.invoke(prompt.format(
    linguagem="Python",
    comando="Crie uma função onde eu faço uma requisição numa URL com o requests"
))   

In [19]:
print(resp_llm.content)

```python
import requests

def make_request(url):
  """Faz uma requisição GET para a URL fornecida e retorna a resposta.

  Args:
    url: A URL para a qual fazer a requisição.

  Returns:
    A resposta da requisição.
  """

  response = requests.get(url)
  return response
```


## Chat Prompt Template

Muito parecido com o Prompt Template, mas com a diferença de que ele é mais voltado para a criação de prompts para conversas em si. Você passa a mensagem e o tipo de papel que o chatbot ou o usuário irá desempenhar.

Tipos de papéis (roles) disponíveis:
- Human: Simulando resposta de um ser humano.
- System: Mensagem que geralmente vem como instrução geral.
- IA: Resposta de um chatbot.

In [8]:
from langchain_core.prompts import ChatPromptTemplate

messages = [
    ("system", "Você é um ótimo cozinheiro e quero que você se comporte e dê respostas apenas nos ingredientes ({ingredientes}) fornecido pelo usuário."),
    ("human", "Dado {ingredientes}, crie 2 receitas detalhando como fazer.")
]

chat_prompt = ChatPromptTemplate.from_messages(messages=messages)
print(chat_prompt.format_messages(ingredientes="molho de tomate, cebola e massa"))

[SystemMessage(content='Você é um ótimo cozinheiro e quero que você se comporte e dê respostas apenas nos ingredientes (molho de tomate, cebola e massa) fornecido pelo usuário.'), HumanMessage(content='Dado molho de tomate, cebola e massa, crie 2 receitas detalhando como fazer.')]


## Few Shot Prompt Template

Neste tipo de prompt, nós podemos passar exemplos de como a LLM deve completar o texto.

In [28]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

exemplos = [
    {
        "sentença":"O filme foi bom",
        "resposta": "Reação: Positiva"
    },
    {
        "sentença":"O filme não foi ruim e nem bom, mas poderia ser melhor",
        "resposta": "Reação: Neutra"
    },
    {
        "sentença":"Não acredito que desperdicei tempo e dinheiro para assistir esse filme",
        "resposta": "Reação: Negativa"
    },
]

exemplo_prompt = PromptTemplate(
    template="{sentença}\n{resposta}\n",
    input_variables=["sentença", "resposta"]
)

few_shot_prompt = FewShotPromptTemplate(
    examples=exemplos,
    example_prompt=exemplo_prompt,
    suffix="sentença: {sentença}",
    input_variables=["sentença"],
)

In [29]:
print(few_shot_prompt.format(sentença="O filme é longo, mas eu gostei. Valeu muito a pena"))

O filme foi bom
Reação: Positiva


O filme não foi ruim e nem bom, mas poderia ser melhor
Reação: Neutra


Não acredito que desperdicei tempo e dinheiro para assistir esse filme
Reação: Negativa


sentença: O filme é longo, mas eu gostei. Valeu muito a pena


In [33]:
resp_llm = llm.invoke(few_shot_prompt.format(sentença="O filme é longo, mas eu gostei. Valeu muito a pena"))
print(resp_llm.content)

Reação: Positiva
